In [1]:
import numpy as np
import sympy as sp
from sympy import symbols, simplify, expand, factor, apart, solve, Poly, init_printing
from sympy.abc import s, t

# Habilita a renderização LaTeX bonita no Colab
init_printing()

# Definir símbolos
Ra, La, Kt, Kb, Jm, Bm, Va, Ia, omega, Ea = symbols('R_a L_a K_t K_b J_m B_m V_a I_a omega E_a', real=True, positive=True)

print("Bibliotecas importadas e símbolos definidos com sucesso.")

Bibliotecas importadas e símbolos definidos com sucesso.


## Tarefa 1: Derivação da Função de Transferência $G(s)$

O sistema é regido pelas seguintes equações diferenciais e constitutivas:

**1. Parte Elétrica:**
$$V_a(t) - E_a(t) = R_a I_a(t) + L_a \frac{dI_a(t)}{dt}$$


**2. Parte Mecânica:**
$$T_m(t) = J_m \frac{d\omega(t)}{dt} + B_m \omega(t)$$

**3. Relações de Acoplamento:**
$$T_m(t) = K_t I_a(t)$$
$$E_a(t) = K_b \omega(t)$$

O objetivo é encontrar $G(s) = \frac{\omega(s)}{V_a(s)}$.

In [2]:
# --- Derivação da Função de Transferência ---

# Definindo o numerador simbólico (Ganho direto)
numerator_sym = Kt

# Definindo o denominador simbólico a partir das equações do sistema
# D(s) = (Ra + La*s)(Jm*s + Bm) + Kb*Kt
denominator_sym = Kb * Kt + (Ra + La * s) * (Jm * s + Bm)

# Expandir o denominador para ver os coeficientes
denominator_expanded = expand(denominator_sym)

# Exibir a Função de Transferência Simbólica
G_s_symbolic = numerator_sym / denominator_expanded

print("Função de Transferência Simbólica G(s):")
display(G_s_symbolic)

# Coletar coeficientes do polinômio do denominador em s
denominator_poly = Poly(denominator_expanded, s)
coeffs = denominator_poly.all_coeffs()

print("\nCoeficientes do Denominador (s^2, s^1, s^0):")
display(coeffs)

Função de Transferência Simbólica G(s):


                     Kₜ                      
─────────────────────────────────────────────
                         2                   
Bₘ⋅Lₐ⋅s + Bₘ⋅Rₐ + Jₘ⋅Lₐ⋅s  + Jₘ⋅Rₐ⋅s + K_b⋅Kₜ


Coeficientes do Denominador (s^2, s^1, s^0):


[Jₘ⋅Lₐ, Bₘ⋅Lₐ + Jₘ⋅Rₐ, Bₘ⋅Rₐ + K_b⋅Kₜ]

## Tarefa 2 e 3: Análise Numérica - Motor SEM Carga

Nesta seção, substituímos os valores físicos dos parâmetros do motor operando livremente (sem carga inercial acoplada).

In [3]:
# --- Parâmetros fornecidos ---
Ra_val = 7.75   # Ohm
La_val = 4.1    # H
Kt_val = 0.0131 # Nm/A
Kb_val = 0.0131 # V*s/rad
J_val  = 5.2e-7 # kg*m² (Rotor apenas)
B_val  = 2.7e-7 # Nm*s/rad (Atrito interno)

# Sem carga: Jm = J, Bm = B
Jm_sem = J_val
Bm_sem = B_val

# Cálculo dos Coeficientes Numéricos (a2*s^2 + a1*s + a0)
a2 = La_val * Jm_sem
a1 = La_val * Bm_sem + Ra_val * Jm_sem
a0 = Ra_val * Bm_sem + Kt_val * Kb_val

print(f"Coeficientes Sem Carga:\n a2={a2:.6e}, a1={a1:.6e}, a0={a0:.6e}")

# Normalização (para o formato s^2 + b1*s + b0)
b1 = a1 / a2
b0 = a0 / a2
K_gain = Kt_val / a2

print("\nPolinômio Característico Normalizado:")
print(f"s² + {b1:.4f}s + {b0:.4f}")

# --- Cálculo dos Polos (Raízes) ---
discriminant = b1**2 - 4*b0

if discriminant >= 0:
    p1 = (-b1 + np.sqrt(discriminant)) / 2
    p2 = (-b1 - np.sqrt(discriminant)) / 2
    tipo_resp = "Superamortecido (Polos Reais)"
else:
    real_part = -b1 / 2
    imag_part = np.sqrt(-discriminant) / 2
    p1 = complex(real_part, imag_part)
    p2 = complex(real_part, -imag_part)
    tipo_resp = "Subamortecido (Polos Complexos)"

print("\n--- Análise de Polos (Sem Carga) ---")
print(f"Tipo de Resposta: {tipo_resp}")
print(f"Polo 1: {p1}")
print(f"Polo 2: {p2}")

Coeficientes Sem Carga:
 a2=2.132000e-06, a1=5.137000e-06, a0=1.737025e-04

Polinômio Característico Normalizado:
s² + 2.4095s + 81.4740

--- Análise de Polos (Sem Carga) ---
Tipo de Resposta: Subamortecido (Polos Complexos)
Polo 1: (-1.204737335834897+8.945533860911327j)
Polo 2: (-1.204737335834897-8.945533860911327j)


## Tarefa 4 e 5: Análise Numérica - Motor COM Carga

Agora adicionamos a inércia e o atrito da carga ao eixo do motor.
$$J_{total} = J_{motor} + J_{carga}$$
$$B_{total} = B_{motor} + B_{carga}$$

In [4]:
# Parâmetros da Carga
Jc_val = 520e-7  # kg*m²
Bc_val = 27e-7   # Nm*s/rad

# Com carga: Somam-se as inércias e atritos
Jm_com = J_val + Jc_val
Bm_com = B_val + Bc_val

# Cálculo dos Coeficientes (Com Carga)
a2_c = La_val * Jm_com
a1_c = La_val * Bm_com + Ra_val * Jm_com
a0_c = Ra_val * Bm_com + Kt_val * Kb_val

# Normalização
b1_c = a1_c / a2_c
b0_c = a0_c / a2_c
K_gain_c = Kt_val / a2_c

print(f"Coeficientes Com Carga:\n a2={a2_c:.6e}, a1={a1_c:.6e}, a0={a0_c:.6e}")
print("\nPolinômio Característico Normalizado (Com Carga):")
print(f"s² + {b1_c:.4f}s + {b0_c:.4f}")

# --- Cálculo dos Polos (Com Carga) ---
discriminant_c = b1_c**2 - 4*b0_c

if discriminant_c >= 0:
    p1_c = (-b1_c + np.sqrt(discriminant_c)) / 2
    p2_c = (-b1_c - np.sqrt(discriminant_c)) / 2
    tipo_resp_c = "Superamortecido (Polos Reais)"
else:
    real_part_c = -b1_c / 2
    imag_part_c = np.sqrt(-discriminant_c) / 2
    p1_c = complex(real_part_c, imag_part_c)
    p2_c = complex(real_part_c, -imag_part_c)
    tipo_resp_c = "Subamortecido (Polos Complexos)"

print("\n--- Análise de Polos (Com Carga) ---")
print(f"Tipo de Resposta: {tipo_resp_c}")
print(f"Polo 1: {p1_c}")
print(f"Polo 2: {p2_c}")

Coeficientes Com Carga:
 a2=2.153320e-04, a1=4.192070e-04, a0=1.946275e-04

Polinômio Característico Normalizado (Com Carga):
s² + 1.9468s + 0.9038

--- Análise de Polos (Com Carga) ---
Tipo de Resposta: Superamortecido (Polos Reais)
Polo 1: -0.7644638005050685
Polo 2: -1.1823299876917628


## Tarefa 6: Comparação Final

Abaixo comparamos como o aumento de inércia e atrito afetou a dinâmica do sistema.

In [5]:
# Comparação de Inércia
ratio_J = Jm_com / Jm_sem
print(f"Aumento de Inércia: {ratio_J:.2f} vezes")

print("\nComparativo de Polos:")
print(f"Sem Carga: {p1:.4f}, {p2:.4f}")
print(f"Com Carga: {p1_c:.4f}, {p2_c:.4f}")

print("\nConclusão:")
if abs(np.real(p1_c)) < abs(np.real(p1)):
    print("Os polos se aproximaram da origem (eixo imaginário).")
    print("Isso indica que o sistema com carga é MAIS LENTO (maior constante de tempo).")
else:
    print("O sistema ficou mais rápido.")

# Verificação de estabilidade
if np.real(p1_c) < 0 and np.real(p2_c) < 0:
    print("O sistema permanece ESTÁVEL.")

Aumento de Inércia: 101.00 vezes

Comparativo de Polos:
Sem Carga: -1.2047+8.9455j, -1.2047-8.9455j
Com Carga: -0.7645, -1.1823

Conclusão:
Os polos se aproximaram da origem (eixo imaginário).
Isso indica que o sistema com carga é MAIS LENTO (maior constante de tempo).
O sistema permanece ESTÁVEL.
